In [1]:
import sys
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
import csv
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.sparse import hstack

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Dropout, Activation , GRU
from keras.layers import Embedding, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D, Bidirectional
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import LSTM, Convolution1D, Flatten, Dropout, Dense
import re
import sys
from nltk.corpus import stopwords
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Using TensorFlow backend.


In [2]:
!pip install PyDrive

    100% |████████████████████████████████| 993kB 22.3MB/s 
  Running setup.py bdist_wheel for PyDrive ... - \ done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
csvfile = drive.CreateFile({'id':'1QptRDzH7K5xgfsnA4_TThO6F0b0ccbjK'})

In [0]:
csvfile.GetContentFile('Data.csv')

In [0]:
data=pd.read_csv('Data.csv')

In [8]:
data.head()

,id,title,motion,manual_motion,govt_opp_motion,motion_party_affiliation,utt1,utt2,utt3,utt4,utt5,manual_speech,vote_speech,party_affiliation,name,rebellion
0,1,Doctor Assisted Dying,That leave be given to bring in a Bill to enab...,1,1,Lab,I trust that the House will not feel that ther...,If one considers the argument from a religious...,I accept that we shall all face death at one t...,"When we listen to my hon. Friend's case, we mu...",I do not think that we want that sort of socie...,0,0,Lab,Kevin McNamara,3.2
1,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,The Secretary State talks of the lack of fuss ...,The right hon. Gentleman may not have heard fu...,North Tayside.,The Secretary of State's geography is terrible.,Would not the £40 million gap in the assumptio...,0,0,SNP,John Swinney,0.4
2,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,May I assure the right hon. Gentleman that he ...,Will the hon. Gentleman clarify what he meant ...,NaN,NaN,NaN,1,1,Lab,John McAllion,4.5
3,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,"Is the hon. Gentleman aware that, ever since t...","I rise as one who, in the past, has not been a...",It would be inappropriate to allow the debate ...,"Why are the problems of so many of our people,...",Today's debate is the first of this sort under...,0,1,Lab,Ian Davidson,3.1
4,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,The hon. Gentleman will notice that the Childr...,I thank the hon. Gentleman for his generosity ...,NaN,NaN,NaN,1,1,Lab,Maria Fyfe,2.3


In [0]:
n_gram_min, n_gram_max = 1, 3

In [0]:
model='2b'

In [0]:
sentiment_label='vote'

In [0]:
features = 3

In [0]:
hansard_stopwords = ({'friend', 'honourable', 'hon', 'gentleman', 'lady', 
					  'give', 'way', 'house', 'amendment', 'beg', 'move', 
					  'member', 'question', 'green', 'white', 'paper', 
					  'bill', 'statement', 'government', 'prime', 'minister', 
					  'opposition', 'party', 'mr', 'rose'})
sklhansard_stopwords = text.ENGLISH_STOP_WORDS.union(hansard_stopwords)

In [0]:
parties =  ['Con', 'Lab', 'LD', 'SNP', 'PC','SDLP', 'DUP', 'UUP', 'Green', 
			'Ind', 'IndCon', 'IndUU']
motion_parties = ['Con', 'Lab', 'LD', 'SNP', 'DUP']

n_gram_min, n_gram_max = 1, 3


In [0]:
vectorizer = TfidfVectorizer(min_df=5, max_df = 1.0, 
	ngram_range=(n_gram_min,n_gram_max), sublinear_tf=True, use_idf =True, 
	stop_words=sklhansard_stopwords)
SVM = LinearSVC()
MLP = MLPClassifier(solver='lbfgs', max_iter=200, alpha=1e-5, hidden_layer_sizes=(100,), random_state=1)

In [0]:
folds_no = 10
kf = StratifiedKFold(n_splits=folds_no, shuffle=True)

In [0]:
data=pd.read_csv('Data.csv')

In [0]:
data["utt1"].fillna(" ", inplace=True)
data["utt2"].fillna(" ", inplace=True)
data["utt3"].fillna(" ", inplace=True)
data["utt4"].fillna(" ", inplace=True)
data["utt5"].fillna(" ", inplace=True)

In [0]:
for i in data.index:
    data.at[i,'speech'] = data.at[i,'utt1'] +" " + data.at[i,'utt2'] +" " + data.at[i,'utt3'] +" " + data.at[i,'utt4'] +" " + data.at[i,'utt5'] 

In [19]:
data

,id,title,motion,manual_motion,govt_opp_motion,motion_party_affiliation,utt1,utt2,utt3,utt4,utt5,manual_speech,vote_speech,party_affiliation,name,rebellion,speech
0,1,Doctor Assisted Dying,That leave be given to bring in a Bill to enab...,1,1,Lab,I trust that the House will not feel that ther...,If one considers the argument from a religious...,I accept that we shall all face death at one t...,"When we listen to my hon. Friend's case, we mu...",I do not think that we want that sort of socie...,0,0,Lab,Kevin McNamara,3.2,I trust that the House will not feel that ther...
1,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,The Secretary State talks of the lack of fuss ...,The right hon. Gentleman may not have heard fu...,North Tayside.,The Secretary of State's geography is terrible.,Would not the £40 million gap in the assumptio...,0,0,SNP,John Swinney,0.4,The Secretary State talks of the lack of fuss ...
2,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,May I assure the right hon. Gentleman that he ...,Will the hon. Gentleman clarify what he meant ...,,,,1,1,Lab,John McAllion,4.5,May I assure the right hon. Gentleman that he ...
3,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,"Is the hon. Gentleman aware that, ever since t...","I rise as one who, in the past, has not been a...",It would be inappropriate to allow the debate ...,"Why are the problems of so many of our people,...",Today's debate is the first of this sort under...,0,1,Lab,Ian Davidson,3.1,"Is the hon. Gentleman aware that, ever since t..."
4,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,The hon. Gentleman will notice that the Childr...,I thank the hon. Gentleman for his generosity ...,,,,1,1,Lab,Maria Fyfe,2.3,The hon. Gentleman will notice that the Childr...
5,3,Orders of the Day — Income Tax,That an humble Address be presented to Her Maj...,0,0,Con,"Does my hon. Friend agree that, whereas last w...",I follow on from what my hon. Friend the Membe...,"That is correct, but PEPs were not.",There was a penalty for withdrawing from PEPs ...,"My hon. Friend makes a valid point, which I wa...",1,1,Con,Nick St. Aubyn,0.6,"Does my hon. Friend agree that, whereas last w..."
6,3,Orders of the Day — Income Tax,That an humble Address be presented to Her Maj...,0,0,Con,Does my hon. Friend agree that part of the dis...,,,,,1,1,Con,Nick Hawkins,0.1,Does my hon. Friend agree that part of the dis...
7,3,Orders of the Day — Income Tax,That an humble Address be presented to Her Maj...,0,0,Con,"First, I must declare an interest as the chair...",I cannot understand how the new scheme will be...,I want to highlight some mechanical problems. ...,I am greatly relieved that the Government have...,The change does not meet the Government's orig...,1,1,Con,Howard Flight,0.3,"First, I must declare an interest as the chair..."
8,4,Orders of the Day – Police,That the Police Grant Report (England and Wale...,1,1,Lab,My hon. Friend mentions Staffordshire and the ...,,,,,0,1,Lab,George Stevenson,1.2,My hon. Friend mentions Staffordshire and the ...
9,4,Orders of the Day – Police,That the Police Grant Report (England and Wale...,1,1,Lab,My right hon. Friend probably heard the interv...,,,,,0,0,Con,John Randall,1.3,My right hon. Friend probably heard the interv...


In [0]:
deb_id=data['id']

In [0]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
data["party_feat"] = lb_make.fit_transform(data["party_affiliation"])

lb_mp = LabelEncoder()
data["motion_party_feat"] = lb_mp.fit_transform(data["motion_party_affiliation"])


In [22]:
data

,id,title,motion,manual_motion,govt_opp_motion,motion_party_affiliation,utt1,utt2,utt3,utt4,utt5,manual_speech,vote_speech,party_affiliation,name,rebellion,speech,party_feat,motion_party_feat
0,1,Doctor Assisted Dying,That leave be given to bring in a Bill to enab...,1,1,Lab,I trust that the House will not feel that ther...,If one considers the argument from a religious...,I accept that we shall all face death at one t...,"When we listen to my hon. Friend's case, we mu...",I do not think that we want that sort of socie...,0,0,Lab,Kevin McNamara,3.2,I trust that the House will not feel that ther...,8,3
1,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,The Secretary State talks of the lack of fuss ...,The right hon. Gentleman may not have heard fu...,North Tayside.,The Secretary of State's geography is terrible.,Would not the £40 million gap in the assumptio...,0,0,SNP,John Swinney,0.4,The Secretary State talks of the lack of fuss ...,11,3
2,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,May I assure the right hon. Gentleman that he ...,Will the hon. Gentleman clarify what he meant ...,,,,1,1,Lab,John McAllion,4.5,May I assure the right hon. Gentleman that he ...,8,3
3,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,"Is the hon. Gentleman aware that, ever since t...","I rise as one who, in the past, has not been a...",It would be inappropriate to allow the debate ...,"Why are the problems of so many of our people,...",Today's debate is the first of this sort under...,0,1,Lab,Ian Davidson,3.1,"Is the hon. Gentleman aware that, ever since t...",8,3
4,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,The hon. Gentleman will notice that the Childr...,I thank the hon. Gentleman for his generosity ...,,,,1,1,Lab,Maria Fyfe,2.3,The hon. Gentleman will notice that the Childr...,8,3
5,3,Orders of the Day — Income Tax,That an humble Address be presented to Her Maj...,0,0,Con,"Does my hon. Friend agree that, whereas last w...",I follow on from what my hon. Friend the Membe...,"That is correct, but PEPs were not.",There was a penalty for withdrawing from PEPs ...,"My hon. Friend makes a valid point, which I wa...",1,1,Con,Nick St. Aubyn,0.6,"Does my hon. Friend agree that, whereas last w...",0,0
6,3,Orders of the Day — Income Tax,That an humble Address be presented to Her Maj...,0,0,Con,Does my hon. Friend agree that part of the dis...,,,,,1,1,Con,Nick Hawkins,0.1,Does my hon. Friend agree that part of the dis...,0,0
7,3,Orders of the Day — Income Tax,That an humble Address be presented to Her Maj...,0,0,Con,"First, I must declare an interest as the chair...",I cannot understand how the new scheme will be...,I want to highlight some mechanical problems. ...,I am greatly relieved that the Government have...,The change does not meet the Government's orig...,1,1,Con,Howard Flight,0.3,"First, I must declare an interest as the chair...",0,0
8,4,Orders of the Day – Police,That the Police Grant Report (England and Wale...,1,1,Lab,My hon. Friend mentions Staffordshire and the ...,,,,,0,1,Lab,George Stevenson,1.2,My hon. Friend mentions Staffordshire and the ...,8,3
9,4,Orders of the Day – Police,That the Police Grant Report (England and Wale...,1,1,Lab,My right hon. Friend probably heard the interv...,,,,,0,0,Con,John Randall,1.3,My right hon. Friend probably heard the interv...,0,3


In [0]:
num_data=data[['id','party_feat','motion_party_feat','manual_motion']]

In [0]:
num_data = np.array(num_data)

In [25]:
num_data

array([[  1,   8,   3,   1],
       [  2,  11,   3,   1],
       [  2,   8,   3,   1],
       ...,
       [129,   1,   4,   0],
       [129,  11,   4,   0],
       [129,   0,   4,   0]])

In [0]:
y_speech=data['vote_speech']

In [0]:
y_speech  = np.array(y_speech)

In [28]:
y_speech

array([0, 0, 1, ..., 0, 1, 0])

In [0]:
X_motions=[]
y_motions=[]

In [0]:
X_motions = data['speech']
y_motions = data['vote_speech']

In [0]:
# for i in data.index:
#     motion = data.at[i,'speech']
#     idval= data.at[i,'id']
#     if len(X_motions) < idval:
#         X_motions.append(motion)
#         y_motions.append(data.at[i,'manual_motion'])

In [34]:
len(X_motions)

1251

In [0]:
# for i in range(len(X_motions)):
#     X_motions.append(X_motions[i])
#     y_motions.append(y_motions[i])

In [36]:
len(X_motions)

1251

In [37]:
X_motions

0       I trust that the House will not feel that ther...
1       The Secretary State talks of the lack of fuss ...
2       May I assure the right hon. Gentleman that he ...
3       Is the hon. Gentleman aware that, ever since t...
4       The hon. Gentleman will notice that the Childr...
5       Does my hon. Friend agree that, whereas last w...
6       Does my hon. Friend agree that part of the dis...
7       First, I must declare an interest as the chair...
8       My hon. Friend mentions Staffordshire and the ...
9       My right hon. Friend probably heard the interv...
10      Does my right hon. Friend agree that the issue...
11      The Minister will be aware that the Gloucester...
12      I welcome what my right hon. Friend has done, ...
13      Is the right hon. Gentleman aware of the publi...
14      I am grateful to the Secretary of State for gi...
15      On the point about local communities supportin...
16      Barnsley council has this year received its hi...
17      Having

In [38]:
len(y_motions)

1251

In [0]:
y_motions = np.array(y_motions)

In [0]:
ids_predictions_list = []
ids_predictions_dict = {}

In [0]:
SVM_motion_acc, SVM_motion_pres, SVM_motion_rec, SVM_motion_f1 = 0, 0, 0, 0
MLP_motion_acc, MLP_motion_pres, MLP_motion_rec, MLP_motion_f1 = 0, 0, 0, 0
DL_motion_acc, DL_motion_pres, DL_motion_rec, DL_motion_f1 = 0, 0, 0, 0


In [0]:
for train_index, test_index in kf.split(X_motions, y_motions):
    #X_motions_train, X_motions_test= X_motions[train_index], X_motions[test_index]
    #y_motions_train, y_motions_test= y_motions[train_index], y_motions[test_index]
    X_motions_train = [X_motions[i] for i in train_index]
    X_motions_test = [X_motions[i] for i in test_index]
    y_motions_train, y_motions_test= y_motions[train_index], y_motions[test_index]
    
    motions_train_corpus = vectorizer.fit_transform(X_motions_train)
    motions_test_corpus = vectorizer.transform(X_motions_test) 
    SVM.fit(motions_train_corpus,y_motions_train)
    MLP.fit(motions_train_corpus,y_motions_train)
    print("Size of train corpus is ", motions_train_corpus.shape)
    print("Size of test corpus is ", motions_test_corpus.shape)
    SVM_motions_predict = SVM.predict(motions_test_corpus)
    MLP_motions_predict = MLP.predict(motions_test_corpus)
    

    print("SVM acc is ", accuracy_score(y_motions_test, SVM_motions_predict), " F1 score is ",f1_score(y_motions_test, SVM_motions_predict) )
    print("MLP acc is ", accuracy_score(y_motions_test, MLP_motions_predict), " F1 score is ",f1_score(y_motions_test, MLP_motions_predict))

    SVM_motion_acc = SVM_motion_acc+accuracy_score(y_motions_test, SVM_motions_predict)
    SVM_motion_pres = SVM_motion_pres+precision_score(y_motions_test, SVM_motions_predict)
    SVM_motion_rec = SVM_motion_rec+recall_score(y_motions_test, SVM_motions_predict)
    SVM_motion_f1 = SVM_motion_f1+f1_score(y_motions_test, SVM_motions_predict)

    MLP_motion_acc = MLP_motion_acc+accuracy_score(y_motions_test, MLP_motions_predict)
    MLP_motion_pres = MLP_motion_pres+precision_score(y_motions_test, MLP_motions_predict)
    MLP_motion_rec = MLP_motion_rec+recall_score(y_motions_test, MLP_motions_predict)
    MLP_motion_f1 = MLP_motion_f1+f1_score(y_motions_test, MLP_motions_predict)


Size of train corpus is  (1125, 3021)
Size of test corpus is  (126, 3021)
SVM acc is  0.6587301587301587  F1 score is  0.7261146496815285
MLP acc is  0.6825396825396826  F1 score is  0.7333333333333334
Size of train corpus is  (1125, 3032)
Size of test corpus is  (126, 3032)
SVM acc is  0.6428571428571429  F1 score is  0.6896551724137931
MLP acc is  0.6746031746031746  F1 score is  0.7050359712230216
Size of train corpus is  (1125, 3027)
Size of test corpus is  (126, 3027)
SVM acc is  0.6587301587301587  F1 score is  0.6950354609929078
MLP acc is  0.626984126984127  F1 score is  0.6618705035971223
Size of train corpus is  (1126, 2972)
Size of test corpus is  (125, 2972)
SVM acc is  0.6  F1 score is  0.6753246753246752
MLP acc is  0.584  F1 score is  0.6623376623376623
Size of train corpus is  (1126, 3069)
Size of test corpus is  (125, 3069)
SVM acc is  0.64  F1 score is  0.6808510638297872
MLP acc is  0.632  F1 score is  0.6805555555555556
Size of train corpus is  (1126, 3017)
Size of 

In [0]:
MLP_motion_f1/10

0.6779111492671233

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer

In [0]:
y_motions=np.array(y_motions)

In [0]:
#DL based model
for train_index, test_index in kf.split(X_motions, y_motions):
    #X_motions_train, X_motions_test= X_motions[train_index], X_motions[test_index]
    #y_motions_train, y_motions_test= y_motions[train_index], y_motions[test_index]
    X_motions_train = [X_motions[i] for i in train_index]
    X_motions_test = [X_motions[i] for i in test_index]
    y_motions_train, y_motions_test= y_motions[train_index], y_motions[test_index]
    

    
    motions_train_corpus = vectorizer.fit_transform(X_motions_train)
    motions_test_corpus = vectorizer.transform(X_motions_test) 
    
    motions_train_corpus= hstack([motions_train_corpus, num_data[train_index]])
    motions_test_corpus = hstack([ motions_test_corpus, num_data[test_index]])
    
    #SVM.fit(motions_train_corpus,y_motions_train)
    #MLP.fit(motions_train_corpus,y_motions_train)
    print("Size of train corpus is ", motions_train_corpus.shape)
    print("Size of test corpus is ", motions_test_corpus.shape)
    #SVM_motions_predict = SVM.predict(motions_test_corpus)
    #MLP_motions_predict = MLP.predict(motions_test_corpus)
    
#     transformer = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=False)
#     motions_train_corpus = transformer.fit_transform(motions_train_corpus)
#     motions_test_corpus=transformer.transform(motions_test_corpus)
    
    
    #print("SVM acc is ", accuracy_score(y_motions_test, SVM_motions_predict), " F1 score is ",f1_score(y_motions_test, SVM_motions_predict) )
    #print("MLP acc is ", accuracy_score(y_motions_test, MLP_motions_predict), " F1 score is ",f1_score(y_motions_test, MLP_motions_predict))

#     SVM_motion_acc = SVM_motion_acc+accuracy_score(y_motions_test, SVM_motions_predict)
#     SVM_motion_pres = SVM_motion_pres+precision_score(y_motions_test, SVM_motions_predict)
#     SVM_motion_rec = SVM_motion_rec+recall_score(y_motions_test, SVM_motions_predict)
#     SVM_motion_f1 = SVM_motion_f1+f1_score(y_motions_test, SVM_motions_predict)

#     MLP_motion_acc = MLP_motion_acc+accuracy_score(y_motions_test, MLP_motions_predict)
#     MLP_motion_pres = MLP_motion_pres+precision_score(y_motions_test, MLP_motions_predict)
#     MLP_motion_rec = MLP_motion_rec+recall_score(y_motions_test, MLP_motions_predict)
#     MLP_motion_f1 = MLP_motion_f1+f1_score(y_motions_test, MLP_motions_predict)
    embed_dim = 128
    lstm_out = 196
    max_features=200
    
    model = Sequential()
    model.add(Embedding(200, 100,input_length = motions_train_corpus.shape[1]))
    model.add(GRU(units=16, name = "gru_1",return_sequences=True))
    model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
    model.add(GRU(units=4, name= "gru_3"))
    model.add(Dense(1, activation='sigmoid',name="dense_1"))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.fit(motions_train_corpus.toarray(), y_motions_train, epochs=8, batch_size=32, verbose=1)
    


Size of train corpus is  (1125, 2973)
Size of test corpus is  (126, 2973)
Epoch 1/8
1125/1125 [==============================] - 105s 93ms/step - loss: 0.6814 - acc: 0.5627
Epoch 2/8
1125/1125 [==============================] - 104s 92ms/step - loss: 0.6186 - acc: 0.6827
Epoch 3/8
1125/1125 [==============================] - 103s 92ms/step - loss: 0.5051 - acc: 0.8071
Epoch 4/8
1125/1125 [==============================] - 104s 92ms/step - loss: 0.4346 - acc: 0.8418
Epoch 5/8
1125/1125 [==============================] - 104s 92ms/step - loss: 0.3640 - acc: 0.8889
Epoch 6/8
1125/1125 [==============================] - 103s 92ms/step - loss: 0.3120 - acc: 0.9093
Epoch 7/8
1125/1125 [==============================] - 104s 93ms/step - loss: 0.2891 - acc: 0.9156
Epoch 8/8
1125/1125 [==============================] - 104s 92ms/step - loss: 0.2782 - acc: 0.9173
Size of train corpus is  (1125, 3062)
Size of test corpus is  (126, 3062)
Epoch 1/8
1125/1125 [==============================] - 110s 

KeyboardInterrupt: 

In [0]:
#DL based model
for train_index, test_index in kf.split(X_motions, y_motions):
    #X_motions_train, X_motions_test= X_motions[train_index], X_motions[test_index]
    #y_motions_train, y_motions_test= y_motions[train_index], y_motions[test_index]
    X_motions_train = [X_motions[i] for i in train_index]
    X_motions_test = [X_motions[i] for i in test_index]
    y_motions_train, y_motions_test= y_motions[train_index], y_motions[test_index]
    
    motions_train_corpus = vectorizer.fit_transform(X_motions_train)
    motions_test_corpus = vectorizer.transform(X_motions_test) 
    #SVM.fit(motions_train_corpus,y_motions_train)
    #MLP.fit(motions_train_corpus,y_motions_train)
    print("Size of train corpus is ", motions_train_corpus.shape)
    print("Size of test corpus is ", motions_test_corpus.shape)
    #SVM_motions_predict = SVM.predict(motions_test_corpus)
    #MLP_motions_predict = MLP.predict(motions_test_corpus)
    
#     transformer = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=False)
#     motions_train_corpus = transformer.fit_transform(motions_train_corpus)
#     motions_test_corpus=transformer.transform(motions_test_corpus)
    
    
    #print("SVM acc is ", accuracy_score(y_motions_test, SVM_motions_predict), " F1 score is ",f1_score(y_motions_test, SVM_motions_predict) )
    #print("MLP acc is ", accuracy_score(y_motions_test, MLP_motions_predict), " F1 score is ",f1_score(y_motions_test, MLP_motions_predict))

#     SVM_motion_acc = SVM_motion_acc+accuracy_score(y_motions_test, SVM_motions_predict)
#     SVM_motion_pres = SVM_motion_pres+precision_score(y_motions_test, SVM_motions_predict)
#     SVM_motion_rec = SVM_motion_rec+recall_score(y_motions_test, SVM_motions_predict)
#     SVM_motion_f1 = SVM_motion_f1+f1_score(y_motions_test, SVM_motions_predict)

#     MLP_motion_acc = MLP_motion_acc+accuracy_score(y_motions_test, MLP_motions_predict)
#     MLP_motion_pres = MLP_motion_pres+precision_score(y_motions_test, MLP_motions_predict)
#     MLP_motion_rec = MLP_motion_rec+recall_score(y_motions_test, MLP_motions_predict)
#     MLP_motion_f1 = MLP_motion_f1+f1_score(y_motions_test, MLP_motions_predict)
    embed_dim = 128
    lstm_out = 196
    max_features=200
    
    model = Sequential()
    model.add(Embedding(200, 100,input_length = motions_train_corpus.shape[1]))
    model.add(GRU(units=16, name = "gru_1",return_sequences=True))
    model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
    model.add(GRU(units=4, name= "gru_3"))
    model.add(Dense(1, activation='sigmoid',name="dense_1"))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.fit(motions_train_corpus, y_motions_train, epochs=8, batch_size=32, verbose=1)
    


Size of train corpus is  (231, 758)
Size of test corpus is  (27, 758)
Epoch 1/8
231/231 [==============================] - 8s 34ms/step - loss: 0.6937 - acc: 0.4762
Epoch 2/8
231/231 [==============================] - 6s 24ms/step - loss: 0.6934 - acc: 0.4935
Epoch 3/8
231/231 [==============================] - 7s 29ms/step - loss: 0.6933 - acc: 0.5195
Epoch 4/8
231/231 [==============================] - 6s 24ms/step - loss: 0.6927 - acc: 0.5195
Epoch 5/8
231/231 [==============================] - 7s 29ms/step - loss: 0.6927 - acc: 0.5195
Epoch 6/8
231/231 [==============================] - 6s 25ms/step - loss: 0.6928 - acc: 0.5195
Epoch 7/8
231/231 [==============================] - 7s 29ms/step - loss: 0.6927 - acc: 0.5195
Epoch 8/8
231/231 [==============================] - 6s 24ms/step - loss: 0.6926 - acc: 0.5195
Size of train corpus is  (231, 748)
Size of test corpus is  (27, 748)
Epoch 1/8
231/231 [==============================] - 7s 30ms/step - loss: 0.6963 - acc: 0.5195
Epoch

In [0]:
#DL based model
for train_index, test_index in kf.split(X_motions, y_motions):
    #X_motions_train, X_motions_test= X_motions[train_index], X_motions[test_index]
    #y_motions_train, y_motions_test= y_motions[train_index], y_motions[test_index]
    X_motions_train = [X_motions[i] for i in train_index]
    X_motions_test = [X_motions[i] for i in test_index]
    y_motions_train, y_motions_test= y_motions[train_index], y_motions[test_index]
    
    motions_train_corpus = vectorizer.fit_transform(X_motions_train)
    motions_test_corpus = vectorizer.transform(X_motions_test) 
    #SVM.fit(motions_train_corpus,y_motions_train)
    #MLP.fit(motions_train_corpus,y_motions_train)
    print("Size of train corpus is ", motions_train_corpus.shape)
    print("Size of test corpus is ", motions_test_corpus.shape)
    #SVM_motions_predict = SVM.predict(motions_test_corpus)
    #MLP_motions_predict = MLP.predict(motions_test_corpus)
    
#     transformer = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=False)
#     motions_train_corpus = transformer.fit_transform(motions_train_corpus)
#     motions_test_corpus=transformer.transform(motions_test_corpus)
    
    
    #print("SVM acc is ", accuracy_score(y_motions_test, SVM_motions_predict), " F1 score is ",f1_score(y_motions_test, SVM_motions_predict) )
    #print("MLP acc is ", accuracy_score(y_motions_test, MLP_motions_predict), " F1 score is ",f1_score(y_motions_test, MLP_motions_predict))

#     SVM_motion_acc = SVM_motion_acc+accuracy_score(y_motions_test, SVM_motions_predict)
#     SVM_motion_pres = SVM_motion_pres+precision_score(y_motions_test, SVM_motions_predict)
#     SVM_motion_rec = SVM_motion_rec+recall_score(y_motions_test, SVM_motions_predict)
#     SVM_motion_f1 = SVM_motion_f1+f1_score(y_motions_test, SVM_motions_predict)

#     MLP_motion_acc = MLP_motion_acc+accuracy_score(y_motions_test, MLP_motions_predict)
#     MLP_motion_pres = MLP_motion_pres+precision_score(y_motions_test, MLP_motions_predict)
#     MLP_motion_rec = MLP_motion_rec+recall_score(y_motions_test, MLP_motions_predict)
#     MLP_motion_f1 = MLP_motion_f1+f1_score(y_motions_test, MLP_motions_predict)
    embed_dim = 128
    lstm_out = 196
    max_features=200
    
    model = Sequential()
    model.add(Embedding(200, 100,input_length = motions_train_corpus.shape[1]))
    model.add(GRU(units=16, name = "gru_1",return_sequences=True))
    model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
    model.add(GRU(units=4, name= "gru_3"))
    model.add(Dense(1, activation='sigmoid',name="dense_1"))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.fit(motions_train_corpus, y_motions_train, epochs=8, batch_size=32, verbose=1)
    


Size of train corpus is  (928, 12099)
Size of test corpus is  (104, 12099)
Epoch 1/8
928/928 [==============================] - 336s 362ms/step - loss: 0.6928 - acc: 0.5194
Epoch 2/8
928/928 [==============================] - 351s 378ms/step - loss: 0.6926 - acc: 0.5194
Epoch 3/8
928/928 [==============================] - 346s 373ms/step - loss: 0.6927 - acc: 0.5194
Epoch 4/8
928/928 [==============================] - 349s 376ms/step - loss: 0.6927 - acc: 0.5194
Epoch 5/8
928/928 [==============================] - 348s 375ms/step - loss: 0.6928 - acc: 0.5194
Epoch 6/8
928/928 [==============================] - 345s 372ms/step - loss: 0.6931 - acc: 0.5194
Epoch 7/8
928/928 [==============================] - 347s 374ms/step - loss: 0.6924 - acc: 0.5194
Epoch 8/8
928/928 [==============================] - 342s 368ms/step - loss: 0.6925 - acc: 0.5194
Size of train corpus is  (928, 12108)
Size of test corpus is  (104, 12108)
Epoch 1/8
928/928 [==============================] - 350s 377ms/st

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [0]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout


def BidLstm(maxlen, max_features, embed_size, embedding_matrix):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix],
                  trainable=False)(inp)
    x = Bidirectional(LSTM(300, return_sequences=True, dropout=0.25,
                           recurrent_dropout=0.25))(x)
    x = Attention(maxlen)(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)

    return model

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [46]:
#DL based model
for train_index, test_index in kf.split(X_motions, y_motions):
    #X_motions_train, X_motions_test= X_motions[train_index], X_motions[test_index]
    #y_motions_train, y_motions_test= y_motions[train_index], y_motions[test_index]
    X_motions_train = [X_motions[i] for i in train_index]
    X_motions_test = [X_motions[i] for i in test_index]
    y_motions_train, y_motions_test= y_motions[train_index], y_motions[test_index]
    

    
    motions_train_corpus = vectorizer.fit_transform(X_motions_train)
    motions_test_corpus = vectorizer.transform(X_motions_test) 
    
    motions_train_corpus= hstack([motions_train_corpus, num_data[train_index]])
    motions_test_corpus = hstack([motions_test_corpus, num_data[test_index]])
    
    #SVM.fit(motions_train_corpus,y_motions_train)
    #MLP.fit(motions_train_corpus,y_motions_train)
    #print("Size of train corpus is ", motions_train_corpus.shape)
    #print("Size of test corpus is ", motions_test_corpus.shape)
    #SVM_motions_predict = SVM.predict(motions_test_corpus)
    #MLP_motions_predict = MLP.predict(motions_test_corpus)
    
#     transformer = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=False)
#     motions_train_corpus = transformer.fit_transform(motions_train_corpus)
#     motions_test_corpus=transformer.transform(motions_test_corpus)
    
    
    #print("SVM acc is ", accuracy_score(y_motions_test, SVM_motions_predict), " F1 score is ",f1_score(y_motions_test, SVM_motions_predict) )
    #print("MLP acc is ", accuracy_score(y_motions_test, MLP_motions_predict), " F1 score is ",f1_score(y_motions_test, MLP_motions_predict))

#     SVM_motion_acc = SVM_motion_acc+accuracy_score(y_motions_test, SVM_motions_predict)
#     SVM_motion_pres = SVM_motion_pres+precision_score(y_motions_test, SVM_motions_predict)
#     SVM_motion_rec = SVM_motion_rec+recall_score(y_motions_test, SVM_motions_predict)
#     SVM_motion_f1 = SVM_motion_f1+f1_score(y_motions_test, SVM_motions_predict)

#     MLP_motion_acc = MLP_motion_acc+accuracy_score(y_motions_test, MLP_motions_predict)
#     MLP_motion_pres = MLP_motion_pres+precision_score(y_motions_test, MLP_motions_predict)
#     MLP_motion_rec = MLP_motion_rec+recall_score(y_motions_test, MLP_motions_predict)
#     MLP_motion_f1 = MLP_motion_f1+f1_score(y_motions_test, MLP_motions_predict)
    embed_size = 128
    lstm_out = 196
    max_features=200000
    maxlen= motions_train_corpus.shape[1]
    
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, input_length=maxlen)(inp)
    x = Bidirectional(LSTM(300, return_sequences=True, dropout=0.25,
                           recurrent_dropout=0.25))(x)
    x = Attention(maxlen)(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    #early = EarlyStopping(monitor="val_loss", mode="min", patience=1)
    
    
#     model = Sequential()
#     model.add(Embedding(200, 100,input_length = motions_train_corpus.shape[1]))
#     model.add(GRU(units=16, name = "gru_1",return_sequences=True))
#     model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
#     model.add(GRU(units=4, name= "gru_3"))
#     model.add(Dense(1, activation='sigmoid',name="dense_1"))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.fit(motions_train_corpus.toarray(), y_motions_train, epochs=10, batch_size=64, verbose=1)
    
    scores = model.evaluate(motions_test_corpus.toarray(), y_motions_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    f1scor=f1_score(y_motions_test, (model.predict_classes(motions_test_corpus.toarray())), average='weighted')
    #print("Values are" ,model.metrics_names, scores)
    print("f2 score is", f1scor)
    
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


Epoch 1/10
1125/1125 [==============================] - 325s 289ms/step - loss: 0.6862 - acc: 0.5600
Epoch 2/10
1125/1125 [==============================] - 319s 283ms/step - loss: 0.6856 - acc: 0.5698
Epoch 3/10
1125/1125 [==============================] - 321s 285ms/step - loss: 0.6844 - acc: 0.5698
Epoch 4/10
1125/1125 [==============================] - 330s 293ms/step - loss: 0.6878 - acc: 0.5698
Epoch 5/10
1125/1125 [==============================] - 325s 289ms/step - loss: 0.6848 - acc: 0.5698
Epoch 6/10
1125/1125 [==============================] - 328s 291ms/step - loss: 0.6861 - acc: 0.5698
Epoch 7/10
1125/1125 [==============================] - 323s 287ms/step - loss: 0.6846 - acc: 0.5698
Epoch 8/10
1125/1125 [==============================] - 324s 288ms/step - loss: 0.6861 - acc: 0.5698
Epoch 9/10
1125/1125 [==============================] - 320s 284ms/step - loss: 0.6853 - acc: 0.5698
Epoch 10/10
1125/1125 [==============================] - 319s 284ms/step - loss: 0.6844 - a

AttributeError: ignored